In [1]:
%config Completer.use_jedi = False
%matplotlib inline
from datasets import load_dataset, Dataset, concatenate_datasets
from IPython.display import clear_output
import csv
import CQ_utils as utils
import HIT_utils as hit
from collections import Counter, defaultdict
from pathlib import Path
import pandas as pd
import json, random
from IPython.core.display import display, HTML
%load_ext autoreload
%autoreload 2
results_root = Path("HIT_results")
path_ambignq = Path("/media/disk1/ksk5693/openQA/ambignq/ambignq_full")  # or ambignq_light
path_ambigqa = Path("/media/disk1/ksk5693/openQA/AmbigQA/")

arg_template = namedtuple('arg_template', 'debug ambigqa')
args = arg_template(False, True)
ambig_cqg_data = AmbigCQGData(logger, args, str(dev_data_path), is_training=False)
tokenized_path = output_dir / "dev_20200201_ambigqa_predictions_BartTokenized_cqg.json"

In [2]:
datafile = "train.json"
json_file = open(path_ambignq / datafile, encoding="utf-8")
train_data = json.load(json_file)
ds_train = utils.json_to_dataset(train_data)

datafile = "dev.json"
json_file = open(path_ambignq / datafile, encoding="utf-8")
dev_data = json.load(json_file)
ds_dev = utils.json_to_dataset(dev_data)

# Add CQ to AmbigNQ

In [3]:
ds_all = concatenate_datasets([ds_train, ds_dev])
ds_all = ds_all.map(hit.preprocess_for_hit)

  0%|          | 0/5921 [00:00<?, ?ex/s]

## 1. Revise Evidently Wrong CQ

In [4]:
def check_validity(ex):
    ans = ex['machine_cq']
    temp = ans.split("--")
    if len(temp) != 2:
        return False
    if "' or '" in temp[-1]:
        options = temp[-1].split(' or ')
    else:
        options = temp[-1].split("', '")
        options = options[:-1] + options[-1].split(", or ")    
    if len(options) != len(ex["dqs"]):
        return False
    return True
invalid_ids = [idx for idx, ex in enumerate(ds_all) if not check_validity(ex)]
ds_ = ds_all.select(invalid_ids)

### 1-1 Prepared annotations

In [5]:
df = pd.read_csv("HIT_results/wrong_results.csv")
cur_idx = 0
mapping = {}
for idx, row in df.iterrows():
    for idx2 in range(9):
        while ds_[cur_idx]["machine_cq"] != row.get(f"Input.machine_cq{idx2}"):
            ds_[cur_idx]["machine_cq"] = "NONE"
            cur_idx += 1
        mapping[cur_idx] = row.get(f"Answer.human_cq{idx2}")

### 1-2 Additional annotations

In [6]:
fault = {
    9: "Could you be more specific--'the name', 'the meaning of the name' or 'the name in eastern orthodox church tradition'?",
    10: "Could you clarify 'what'--'west bank' or 'Gaza Strip'?",    
    11: "Could you clarify 'dumb and dumber 2'--'the Second Dumb and Dumber movie' or 'Dumb and Dumber To'?",    
    12: "Could you be more specific--'who' or 'what'?",    
    13: "Could you be more specific--'where' or 'how many'?",    
    14: "Could you clarify 'start'--'start in new zealand', 'first game', or 'end'?",    
    15: "Could you clarify 'a group'--'a group' or 'called'?",    
    16: "Could you clarify 'who'--'overall' or 'among presidents have died'?",    
    17: "Could you clarifiy 'most own goals'--'in total' or 'in a season'",    
}
mapping.update(fault)
mapping_new = {}
for k, v in mapping.items():
    mapping_new[invalid_ids[k]] = v

### 1-3 Overwrite CQs

In [7]:
def temp(x, idx):
    x["machine_cq"]=mapping_new.get(idx, x["machine_cq"])
    return x
ds_all = ds_all.map(temp, with_indices=True)

  0%|          | 0/5921 [00:00<?, ?ex/s]

## 2. Split data into train and dev

In [8]:
remove_columns = [
    "viewed_doc_titles", "gold_passages", "nouns_by_titles", "candidates",
    "aq_underlined", "aq_", "dqs_", "machine_marked_cq", "concated_dqs_and_answers", 
    "marked_aq", "concated_passages"
]
ds_all = ds_all.map(lambda x: x, remove_columns=remove_columns)

  0%|          | 0/5921 [00:00<?, ?ex/s]

In [9]:
ds_train_ = ds_all.select([i for i in range(len(ds_train))])
ds_dev_ = ds_all.select([i for i in range(len(ds_train), len(ds_train)+len(ds_dev))]) 

## 3. Save datasets

In [15]:
ds_dev_[5]

{'aq': "Where was 'top of the lake' filmed?",
 'dqs': ["Where were the scenes in the women's commune in season 1 of 'top of the lake' filmed?",
  "Where was season 2 of 'top of the lake' filmed?",
  "Where was the original 'top of the lake' filmed except for the women's commune?",
  "Where was 'top of the lake: China Girl' filmed?"],
 'answers': [['Moke Lake', 'Closeburn, Queenstown, South Island'],
  ['Sydney, Australia'],
  ['Queenstown and Glenorchy', 'Queenstown and Glenorchy, in Otago'],
  ['Sydney', 'Sydney, Australia']],
 'orig_idx': 12,
 'machine_cq': "Could you clarify 'the lake'--'were the scenes in the women 's commune in season 1 of', 'season 2 of', 'the original except for the women 's commune', or ': china girl'?"}

In [16]:
train_data_ = utils.augment_json_with_cq(train_data, ds_train_)
dev_data_ = utils.augment_json_with_cq(dev_data, ds_dev_)

with open(path_ambigqa / "data" / "ambigqa" / "train_cq.json", 'w') as f:
    json.dump(train_data_, f)

with open(path_ambigqa / "data" / "ambigqa" / "dev_cq.json", 'w') as f:
    json.dump(dev_data_, f)  

## 4. (DEPRECATED) Prepare re-ranking results

base = path_ambigqa / "psg_sel"

split, ds = "train", ds_train
reranked = base/"reranking_results"/f"ambigqa_{split}_2020.json"
with open(reranked) as f:
    reranked = json.load(f)
selected_reranked = [reranked[ex['orig_idx']] for ex in ds]      
with open(base/"reranking_results"/f"ambigcq_{split}_2020.json", "w") as json_file:
    json.dump(selected_reranked, json_file)
    
split, ds = "dev", ds_dev
reranked = base/"reranking_results"/f"ambigqa_{split}_2020.json"
with open(reranked) as f:
    reranked = json.load(f)
selected_reranked = [reranked[ex['orig_idx']] for ex in ds]      
with open(base/"reranking_results"/f"ambigcq_{split}_2020.json", "w") as json_file:
    json.dump(selected_reranked, json_file)    

In [3]:
ds_dev = load_dataset('json', data_files=path_ambigqa / "data" / "ambigqa" / "dev_cq_light.json")

TypeError: 'PosixPath' object is not iterable

In [15]:
ds_dev

DatasetDict({
    train: Dataset({
        features: ['aq', 'dqs', 'answers', 'viewed_doc_titles', 'gold_passages', 'orig_idx', 'nouns_by_titles', 'candidates', 'aq_underlined', 'aq_', 'dqs_', 'machine_marked_cq', 'machine_cq', 'concated_dqs_and_answers', 'marked_aq', 'concated_passages'],
        num_rows: 1172
    })
})

In [13]:
ds_dev['train']['aq']

['Why did the st louis cardinals move to arizona?',
 'Who is the current chairman of african union commission?',
 'Who won the final hoh big brother 20?',
 'How long do contestants get to answer on jeopardy?',
 'Who took control of the italian government in 1922?',
 "Where was 'top of the lake' filmed?",
 'How many seconds do you have to throw a grenade?',
 "How many jimmy john's are there in the world?",
 'Who defeated the new york jets on the first televised monday night football game?',
 'When is episode 113 of dragon ball super coming out?',
 'Who is the longest serving manager in the premier league?',
 'Who sang i fooled around and fell in love?',
 'Who the female singer on gimme shelter?',
 'When was the commission fir sc/st established?',
 'What do the symbols on the indian flag represent?',
 'Where is the stern of a boat located?',
 'Who is the administrator of the small business administration?',
 'What is the tallest ride at six flags over texas?',
 'Who sang the original do 

In [10]:
ds_dev['train'][0]

{'aq': 'Why did the st louis cardinals move to arizona?',
 'dqs': ['What ability caused the st louis cardinals move to arizona?',
  'What physical issue caused the st louis cardinals move to arizona?',
  'What fan issue caused the st louis cardinals move to arizona?'],
 'answers': [['overall mediocrity of the Cardinals'],
  ['old stadium'],
  ['game attendance to dwindle']],
 'viewed_doc_titles': ['History of the St. Louis Cardinals (NFL)',
  'History of the Arizona Cardinals'],
 'gold_passages': [['History of the St. Louis Cardinals (NFL)',
   'The professional American football team now known as the <b>Arizona Cardinals</b> \npreviously ... During the <b>Cardinals</b>&#39; tenure in <b>St</b>. <b>Louis</b>, they <b>were</b> called the &quot;Big \nRed&quot;, the ... On March 15, 1988, the NFL team owners voted to allow Bidwill to \n<b>move</b> the <b>Cardinals</b> from <b>St</b>. <b>Louis</b> to Tempe, <b>Arizona</b> for the 1988 NFL season.'],
  ['History of the Arizona Cardinals',
 

In [9]:
from transformers import BartTokenizer

In [13]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [16]:
[len(tokenizer.encode(ex["machine_cq"])) for ex in ds_dev]

NameError: name 'ds_dev' is not defined

In [11]:
import transformers

In [12]:
print(transformers.__version__)

3.0.0


In [30]:
def check_validity(ex):
    ans = ex['machine_cq']
    temp = ans.split("--")
    if len(temp) != 2:
        return False
    if "' or '" in temp[-1]:
        options = temp[-1].split(' or ')
    else:
        options = temp[-1].split("', '")
        options = options[:-1] + options[-1].split(", or ")    
    if len(options) != len(ex["dqs"]):
        return False
    return True

In [40]:
df

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Answer.human_cq1,Answer.human_cq2,Answer.human_cq3,Answer.human_cq4,Answer.human_cq5,Answer.human_cq6,Answer.human_cq7,Answer.human_cq8,Approve,Reject
0,3MWOYZD5X6BC55UT38OR2SX7OBCNO8,39P6QDR003RZ9IRNUQ1RZZPYQFHMDS,CQ WRONG,CQ WRONG,"survey, demographics",$1.00,Sun May 29 20:20:05 PDT 2022,1,BatchId:356838;OriginalHitTemplateId:920937243;,25200,...,"Could you clarify 'new album'--'compilation', ...","Could you clarify 'start'--'end', 'commonly st...","Could you clarify 'who'--'the character', 'the...",Could you be more specific--'who holds the rec...,Could you clarify 'begin'--'begin' or 'end'?,Could you clarify 'known'--'most famously' or ...,Could you clarify 'start'--'start' or 'finish'?,Could you clarify 'the pair of hand drums'--'t...,NaN,NaN
1,3H6W48L9GFCKDZF8A9STNBFORSCPWI,39P6QDR003RZ9IRNUQ1RZZPYQFHMDS,CQ WRONG,CQ WRONG,"survey, demographics",$1.00,Sun May 29 20:20:05 PDT 2022,1,BatchId:356838;OriginalHitTemplateId:920937243;,25200,...,Could you clarify 'coins'--'the two-cent piece...,Could you clarify 'start'--'start' or 'finish'?,Could you clarify 'office 2019'--'commercial p...,Could you clarify 'men’s and women’s'--'men’s'...,"Could you be more specific--'how many games', ...",Could you clarify 'located'--'by which river' ...,Could you clarify 'start'--'begin' or 'end'?,Could you clarify 'When'--'which super bowl' o...,NaN,NaN
2,38DCH97KISPJ7RGO3NO3YQBQUSPJQ8,39P6QDR003RZ9IRNUQ1RZZPYQFHMDS,CQ WRONG,CQ WRONG,"survey, demographics",$1.00,Sun May 29 20:20:05 PDT 2022,1,BatchId:356838;OriginalHitTemplateId:920937243;,25200,...,Could you be more specific--'start according t...,Could you clarify 'where'--'through what' or '...,Could you be more specific--'if you are born i...,"Could you clarify 'where'--'what', 'In what', ...",Could you clarify 'superoxide dismutase'--'sod...,Could you clarify 'the brady bunch'--'the brad...,"Could you be more specific-'what was', 'when',...",Could be more specific--'where' or 'when'?,NaN,NaN
3,3CRWSLD92VR6NN9TR79RPD87JL2MO7,39P6QDR003RZ9IRNUQ1RZZPYQFHMDS,CQ WRONG,CQ WRONG,"survey, demographics",$1.00,Sun May 29 20:20:05 PDT 2022,1,BatchId:356838;OriginalHitTemplateId:920937243;,25200,...,"Could you be more specific--'the goonies I', '...",Could you clarify 'most goals'--'what' or 'wha...,Could you clarify 'model'--'whose' or 'what'?,Could you clarify 'alternate ending'--'where c...,Could you be more specific--'in US' or 'in Bri...,"Could you be more specific--'in US', 'in Nethe...",Could you clarify 'start'--'start' or 'end'?,Could you be more specific--'x axis' or 'y axis'?,NaN,NaN
4,3E22YV8GHCTUQ56UI816RS2XYG8NPP,39P6QDR003RZ9IRNUQ1RZZPYQFHMDS,CQ WRONG,CQ WRONG,"survey, demographics",$1.00,Sun May 29 20:20:05 PDT 2022,1,BatchId:356838;OriginalHitTemplateId:920937243;,25200,...,Could you be more specific--'most goals' or 'a...,"Could you be more specific--'from what opera',...",Could you clarify 'what type of tissue'--'what...,Could you clarify 'begin'--'begin' or 'end'?,Could you be more specific--'plasma membrane b...,Could you be more specific--'when' or 'where'?,Could you be more specific--'who' or 'what'?,Could you clarify 'wrote'--'wrote' or 'produced'?,NaN,NaN
5,3MD8CKRQ0AAJJSVOG2D33GLVQERJR9,39P6QDR003RZ9IRNUQ1RZZPYQFHMDS,CQ WRONG,CQ WRONG,"survey, demographics",$1.00,Sun May 29 20:20:05 PDT 2022,1,BatchId:356838;OriginalHitTemplateId:920937243;,25200,...,Could you be more specific--'when' or 'why?,Could you clarify 'nfl record'--'who' or 'what'?,"Could you clarify 'who'--'none', 'that was coa...",Could you clarify 'workers'--'workers' or 'lab...,"Could you clarify 'season 3'--'Season 3', 'Sea...",Could you be more specific--'in Australia' or ...,Could you clarify 'Where'--'Where' or 'what ri...,Could you clarify 'failure'--'failure' or 'bac...,NaN,NaN
6,334ZEL5JYH29LIMYB45D1M7DBAKSOP,39P6QDR003RZ9IRNUQ1RZZPYQFHMDS,CQ WRONG,CQ WRONG,"survey, demogra

In [72]:
for idx in range(9, 18):
    print(idx)
    print(ds_[idx]["aq"])    
    print(ds_[idx]["machine_cq"])
    for i in ds_[idx]["dqs"]:
        print(i)
    print("----------")

9
What is the name of the samaritan woman at the well?
Could you clarify 'the name'--'the meaning of' or 'in eastern orthodox church tradition'?
What's the name of the samaritan woman at the well?
What is the meaning of the name of the samaritan woman at the well?
What is the name of the samaritan woman at the well in Eastern Orthodox Church tradition?
----------
10
What is the west bank and gaza strip?
Could you clarify 'what'--?
What is the west bank ?
What is the Gaza Strip?
----------
11
Who played freda in dumb and dumber 2?
Could you clarify 'played'--'the second movie'?
Who played Freda in the Second Dumb and Dumber movie?
Who played Freda in Dumb and Dumber To?
----------
12
Most runs by a player in single ipl season?
Could you clarify 'runs'--'has scored'?
Who has the most runs by a player in a single IPL season?
What is the most runs a player has scored in a single IPL season?
----------
13
Where was the hindenburg scheduled to fly and how many passengers were on the british 

In [54]:
df.iloc[0]["Input.machine_cq0"]

"Could you clarify 'who'--'when did live'?"

In [11]:
ex['machine_cq'].

"Could you clarify 'the simpsons first air'--'as an animated short on the tracey ullman show' or 'as a half-hour prime time show'?"

In [76]:
sorted(mapping.keys())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152]

In [54]:
df.iloc[0]["Input.machine_cq0"]

"Could you clarify 'who'--'when did live'?"

In [32]:
invalid_ids = [idx for idx, ex in enumerate(ds_) if not check_validity(ex)]

In [34]:
len(invalid_ids)

153

In [35]:
name = "wrong"
ids, num_queries_per_page = invalid_ids, 9
ds_ = ds_all.select(ids).map(hit.preprocess_for_hit)
ds_ = ds_.map(lambda x: x, remove_columns=["viewed_doc_titles", "answers", "orig_idx", "dqs",
                                           "gold_passages", "nouns_by_titles", "aq_", "dqs_", "candidates"])
sharded_sets = [ds_.shard(num_shards=num_queries_per_page, index=i) for i in range(num_queries_per_page)]
ds_ = hit.merge_queries(sharded_sets)
filename = f"data_splits_for_HIT/{name}.csv"
print(f"Save: {filename}")
ds_.to_csv(filename, index=False)

  0%|          | 0/153 [00:00<?, ?ex/s]

  0%|          | 0/153 [00:00<?, ?ex/s]

Save: data_splits_for_HIT/wrong.csv


159092

In [36]:
name = "wrong"
ids, num_queries_per_page = invalid_ids, 9
hit.compose_mturk_html(ds_all, good_examples,
                       csv_file=f"data_splits_for_HIT/{name}.csv",
                       html_template_file=f"{name}_template.html",
                       html_example_file=f"{name}_example.html",
                       basic_html_code=hit.BASIC_HTML_CODE)

False


# Used examples as good examples

In [3]:
# idx, reason, gold_cq.
good_examples = [
    # G0: when did ireland rugby last win the grand slam ?
    {"idx": 28, 
     "subtitle": "No revision required.", 
     "reason": "{option}{ambiguous_part} makes sense.",
    },
    # G1: when did the packers play at camp randall ? 
    {"idx": 88, 
     "subtitle": "Revision required: choose {ambiguous part} from suggested underlined candidates.", 
     "reason": "{option}{ambiguous_part} makes sense.",
     "gold_cq": "Could you clarify '<mark>play</mark>'--'first', 'last , as of 2017'?"
    },
    # G2: who is the current archbishop of los angeles ? 
    {"idx": 259, 
     "subtitle": "Revision required: write {ambiguous part} that is a part of one suggested underlined candidate.", 
     "reason": "The desirable {ambiguous part} is 'current' that is not in suggested underlined candidates. In such cases, you need to write 'current' regardless of suggested candidates.",
     "gold_cq": "<span style='font-family:monospace'>Could you clarify '<mark>current</mark>'--'5th', '4th', or '3rd'?"
    },
    # G3: where does the cumberland river begin and end ?
    {"idx": 85, 
     "subtitle": "Revision required: revise {option_i}.", 
     "reason": "The auto-generated {option_i}s are wrong.",
     "gold_cq": "<span style='font-family:monospace'>Could you clarify '<mark>begin</mark>'--<mark>'begin' or 'end'</mark></span>?"
    },    
    # G4: When did the edwardian era start and end?  
    {"idx": 55, 
     "subtitle": "Revision required: need to read Wikipedia passsages.", 
     "reason": "Provided possible interpretaions are wrong according to Wikipedia Passages.",
     "gold_cq": "<span style='font-family:monospace'>Could you clarify '<mark>start</mark>'--<mark>'start' or 'end'</mark></span>?"
    },  
    # G5: when did the han solo movie come out ?    
    {"idx": 8, 
     "subtitle": "Revision required: need to delete redundant parts of options.", 
     "reason": "<mark>'first premiered' and 'came' are redundant when 'come' is chosen as the {ambiguous part}.</mark> {ambiguous part}{option} makes sense.",
     "gold_cq": "<span style='font-family:monospace'>Could you clarify '<mark>come</mark>'--<mark>'in los angeles' or 'in united states'</mark></span>?"
    },
    # G6: what is the population of rochester new york ?     
    {"idx": 76, 
     "subtitle": "Revision required: when picking one {ambiguous part} is hard.", 
     "reason": "<mark>Choosing either 'rochester new york' or 'the population' as {ambiguous part} makes sense. Therefore, write 'Could you be more specific' without {ambiguous part}. Also, revising {option_i} in the form of 'in 19XX' is correct as it has minimal information enough to be distinguished from other options.</mark>",
     "gold_cq": "<span style='font-family:monospace'><mark>Could you be more specific--'in 2010', 'in 2000', 'in 1990', or 'in 2018'?</mark></span>" 
    },
    # G7: when was the forbidden city opened to the public ?
    {"idx": 64, 
     "subtitle": "No revision required: replacing {ambiguous part} with {option_i} makes sense.", 
     "reason": "Putting {option_i} instead of {ambiguous part} in the ambiguous question makes the question be clarified.",
    },
    # G8: what are the measurements of a full mattress ? 
    {"idx": 86, 
     "subtitle": "Revision required: when picking one {ambiguous part} is hard", 
     "reason": "<mark>Both 'measurement' and 'a full mattress' can be {ambiguous part}.</mark> In such cases, write 'Could you be more specific' without {ambiguous part}.",
     "gold_cq": "<span style='font-family:monospace'><mark>Could you be more specific</mark>'--'standard in inches', 'standard in centimeters', 'xl in inches', or 'xl in centimeters'</span>'?"
    },    
]

# Distribute examples

In [4]:
num_validation_examples = 71
num_examples_to_be_annotated = len(ds_all) - num_validation_examples
num_groups = 10
num_hits_per_group = 65
num_examples_per_hit = 10
num_validation_example_per_hit = 1

num_validation_examples_first = 10 # Free to change. to 71. (allow overlapping with second validation)

# 1st qualification
filter1 = [i for i in range(num_validation_examples_first)] 

# 2nd qualification per group
filter2 = [i for i in range(num_validation_examples-num_hits_per_group, num_validation_examples)] 
filter2_placements = random.choices(range(10), k=num_hits_per_group)

# checksum
assert num_examples_to_be_annotated == num_hits_per_group*(num_examples_per_hit-num_validation_example_per_hit)*num_groups

distribution = {"qualifier": ([i for i in range(num_validation_examples_first)], num_validation_examples_first)}

offset = num_validation_examples
for group_id in range(num_groups):
    total_ids = []
    offset += group_id * (num_hits_per_group * num_examples_per_hit)
    for hit_id in range(num_hits_per_group):
        start_idx = offset + hit_id * (num_examples_per_hit-num_validation_example_per_hit)
        end_idx = offset + (hit_id+1) * (num_examples_per_hit-num_validation_example_per_hit)
        ids = [i for i in range(start_idx, end_idx)]
        ids = ids[:filter2_placements[hit_id]] + [filter2[hit_id]] + ids[filter2_placements[hit_id]:]
        total_ids += ids
    distribution[f"group{group_id}"] = (total_ids, num_examples_per_hit)

In [8]:
for name, info in distribution.items():
    ids, num_queries_per_page = info
    ds_ = ds_all.select(ids).map(hit.preprocess_for_hit)
    ds_ = ds_.map(lambda x: x, remove_columns=["viewed_doc_titles", "answers", "orig_idx", "dqs",
                                               "gold_passages", "nouns_by_titles", "aq_", "dqs_", "candidates"])
    sharded_sets = [ds_.shard(num_shards=num_queries_per_page, index=i) for i in range(num_queries_per_page)]
    ds_ = hit.merge_queries(sharded_sets)
    filename = f"data_splits_for_HIT/{name}.csv"
    print(f"Save: {filename}")
    ds_.to_csv(filename, index=False)

  0%|          | 0/10 [00:00<?, ?ex/s]

  0%|          | 0/10 [00:00<?, ?ex/s]

Save: data_splits_for_HIT/qualifier.csv


  0%|          | 0/650 [00:00<?, ?ex/s]

  0%|          | 0/650 [00:00<?, ?ex/s]

...

  0%|          | 0/650 [00:00<?, ?ex/s]

Save: data_splits_for_HIT/group8.csv


  0%|          | 0/650 [00:00<?, ?ex/s]

  0%|          | 0/650 [00:00<?, ?ex/s]

Save: data_splits_for_HIT/group9.csv


## Write qualification HTML codes

In [9]:
name = "group1"
ids, num_queries_per_page = distribution[name]
hit.compose_mturk_html(ds_all, good_examples,
                       csv_file=f"data_splits_for_HIT/{name}.csv",
                       html_template_file=f"{name}_template.html",
                       html_example_file=f"{name}_example.html",
                       basic_html_code=hit.BASIC_HTML_CODE)

False


In [13]:
len(ds_all)

5921

In [11]:
hit.BASIC_HTML_CODE_FOR_QUALIFICATION

'\n<style>\n.query_box\n{\n   border: 2px solid black;\n   margin-top:0px;\n   margin-right:15px;\n   margin-bottom:0px;\n   margin-left:15px;\n}\n.passage_box\n{\n   border: 1px solid black;\n   margin-top:0px;\n   margin-right:15px;\n   margin-bottom:0px;\n   margin-left:15px;\n}\n.aq_box\n{\n   /* width:300px; */\n   border: 1px solid black;\n   margin-top:15px;\n   margin-right:0px;\n   margin-bottom:0px;\n   margin-left:15px;\n}\n.instruction_title_text_style\n{\n  font-weight: bold;\n  font-size: 15px;\n}\n.raw_text_style\n{\n  font-family: monospace;\n  font-size: 15px;\n}\n.collapsible {\n  background-color: #777;\n  color: white;\n  cursor: pointer;\n  padding: 18px;\n  width: 100%;\n  border: none;\n  text-align: left;\n  outline: none;\n  font-size: 15px;\n}\n\n.active, .collapsible:hover {\n  background-color: #555;\n}\n\n.collapsible:after {\n  content: \'\\002B\';\n  color: white;\n  font-weight: bold;\n  float: right;\n  margin-left: 5px;\n}\n\n.active:after {\n  content

# Used examples in Instructions
* train-0: used in instructions for underlined...  
* train-7: used in instructions for underlined...

In [8]:
# selected_ids = ids_gold
# # selected_ids = ids_silver
# ds_ = ds_train.select(selected_ids).map(hit.preprocess_for_hit)
# ds_ = ds_.map(lambda x: x, remove_columns=["viewed_doc_titles", "answers", "orig_idx", "dqs",
#                                            "gold_passages", "nouns_by_titles", "aq_", "dqs_", "candidates"])
# file = []
# for inst, idx in zip(ds_, ids_gold):
#     file.append(f"<p> {idx} <br>")
#     file.append(inst["marked_aq"] + "<br>")
#     file.append(inst["machine_marked_cq"])
#     file.append("</p>")
# with open("gold_examples.html", 'w') as f:
#     for line in file:
#         f.write(line + "\n")

# selected_ids = ids_silver
# ds_ = ds_train.select(selected_ids).map(hit.preprocess_for_hit)
# ds_ = ds_.map(lambda x: x, remove_columns=["viewed_doc_titles", "answers", "orig_idx", "dqs",
#                                            "gold_passages", "nouns_by_titles", "aq_", "dqs_", "candidates"])        
# file = []
# for inst, idx in zip(ds_, ids_silver):
#     file.append(f"<p> {idx} <br>")
#     file.append(inst["marked_aq"] + "<br>")
#     file.append(inst["machine_marked_cq"])
#     file.append("</p>")
# with open("silver_examples.html", 'w') as f:
#     for line in file:
#         f.write(line + "\n")        
        
        
selected_ids = [i for i in range(300) if i not in ids_gold + ids_silver]
ds_ = ds_train.select(selected_ids).map(hit.preprocess_for_hit)
ds_ = ds_.map(lambda x: x, remove_columns=["viewed_doc_titles", "answers", "orig_idx", "dqs",
                                           "gold_passages", "nouns_by_titles", "aq_", "dqs_", "candidates"])        
file = []
for inst, idx in zip(ds_, selected_ids):
    file.append(f"<p> {idx} <br>")
    file.append(inst["marked_aq"] + "<br>")
    file.append(inst["machine_marked_cq"])
    file.append("</p>")
with open("bad_examples.html", 'w') as f:
    for line in file:
        f.write(line + "\n")              

  0%|          | 0/184 [00:00<?, ?ex/s]

  0%|          | 0/184 [00:00<?, ?ex/s]

In [9]:
%debug

> <ipython-input-6-30d92093050c>(31)<listcomp>()
     29 
     30 
---> 31 selected_ids = [i for i in range(300) if i not in ids_gold + ids_silver]
     32 ds_ = ds_train.select(selected_ids).map(hit.preprocess_for_hit)
     33 ds_ = ds_.map(lambda x: x, remove_columns=["viewed_doc_titles", "answers", "orig_idx", "dqs",

ipdb> s
